In [1]:
import os
op = os.path
opd = op.dirname
opj = op.join
orig_dir = os.getcwd()

In [2]:
os.chdir( opj( opd( opd( orig_dir ) ) ) )

In [3]:
%matplotlib
##
from glob import glob
from pathlib import Path
from time import sleep, time
import yaml
import re
import numpy as np
from matplotlib import pyplot as plt
import cv2
##
from modules.devices.Olympuslx81 import OLYMP
from modules.devices.Prior import PRIOR
from modules.devices.Evolve512 import EVOLVE
###
from tensorflow.keras import models
import tensorflow.keras as tfk

Using matplotlib backend: Qt5Agg


In [4]:
ol = OLYMP()
pr = PRIOR()
ev = EVOLVE()

########### For BF autocontrast is True


In [13]:
mod_fold = 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022/*'
lm = glob(mod_fold)
# print(lm)

## Load the models

In [14]:
lm_sorted = sorted(lm, key= lambda elem : int(re.findall('lev(-?\\d+)', elem)[0]))
print(f'lm_sorted { lm_sorted }')
list_mod = []
t0 = time()
print('loading the models for fast AFML.. ')
for m in lm_sorted:
    list_mod += [models.load_model(m)]
t1 = time()
# print(f'time elapsed for loading the models is { round((t1-t0)/60,1) } min')

lm_sorted ['Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev-6-ep50-bs4_date03-12-2022-23-16', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev-5-ep50-bs4_date03-12-2022-22-28', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev-4-ep50-bs4_date03-12-2022-21-42', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev-3-ep50-bs4_date03-12-2022-20-56', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev-2-ep50-bs4_date03-12-2022-20-10', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev-1-ep50-bs4_date03-12-2022-19-24', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev0-ep50-bs4_date04-12-2022-00-02', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev1-ep50-bs4_date04-12-2022-00-49', 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_dec4th2022\\lev2-ep50-bs4_date04-12-2022-0

In [15]:
def take_pic_posz(i, val, bpp=8, move_type='d', show=False):
    '''
    take the picture at pos posz
    '''
    ol.go_zpos( val, move_type=move_type )            # 797300 
    answ = ol.receive(13)
    print(f'answ 0 {answ}')
    #addr_img = f'uu{i}.tiff'
    addr_img = f'bf_for_optimz{i}.tiff'
    ev.take_pic(addr=addr_img, bpp=bpp, exp_time=100)
#     ev.handle_contrast(True, bpp)
    ev.adapt(bpp)
    if show: plt.imshow(ev.frame, cmap='gray')
    return ev.frame

In [16]:
def make_pred(i, mod=None, thresh=127, show=False):
    '''
    make the prediction and return surface 
    '''
    img = cv2.imread(f'bf_for_optimz{i}.tiff')
    arr = np.array([ img ], dtype=np.float32)/255                        # format array for prediction
    pred = mod.predict(arr)[0]*255
    ##
    ll = len( np.where( pred > thresh )[0] )
    #print(f'll { ll }')
    addr_pred = f'pred_uu{i}.png'
    cv2.imwrite(addr_pred, pred)
    if show:
        plt.figure()
        plt.imshow(pred, cmap='gray')
    return ll

In [17]:
def find_z_optim(z, fill_dic_surf=False):
    '''
    find the optimal focus
    z: z position eg: 785400
    '''
    t0 = time()
    step = 20
    pic0 = take_pic_posz(0, z , bpp=8, move_type='d')
    pic1 = take_pic_posz(1, z + step, bpp=8, move_type='d')
    ldiff = []
    print('--------')
    optim = 0
    max_surf = 0
    # Applying the models.. 
    for num, m in enumerate(list_mod):
        surf0 = make_pred(0, mod=m, thresh=127, show=False)
        surf1 = make_pred(1, mod=m, thresh=127, show=False)
        diff = surf1-surf0
        if max_surf < surf0:
            max_surf = surf0
        ldiff += [[surf0, diff]]
        print(f'for mod {num}, diff is {diff}')
        if fill_dic_surf:
            try:
                dic_surf[num] += [surf0]
            except:
                dic_surf[num] = [surf0]
    print(f'max_surf = { max_surf }')
    for mind,(surf,slope) in enumerate(ldiff):
        if surf > max_surf/5 and mind>0:
            if np.sign(slope) == -np.sign(old_slope):
                if np.abs(slope)< np.abs(old_slope):
                    optim = mind
                else:
                    optim = mind-1
                break
        old_slope = slope
            
    
    print(f'***** optim is {optim} *****')
    t1 = time()
    print(f'time elapsed is { round((t1-t0),1) } s')
    return optim

In [164]:
def plot_curves(lcurves):
    '''
    '''
    lposz, lposz_corr, loptim, lsurf0, ldiff2, z, step = lcurves
    plt.plot(lposz, label='input')
    plt.plot(lposz_corr, label='correction')
    plt.legend()
    print('finished..')
    date = 'dec6th2022'
    plt.savefig(f'meth_13mod_stability_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('optim')
    plt.plot(loptim)
    plt.savefig(f'list_optim_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('surf model 6')
    plt.plot(lsurf0)
    plt.savefig(f'list_mod0_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('diff model 6')
    plt.plot(ldiff2)
    plt.savefig(f'list_ldiff2_zmid{z}_step{step}_{date}.png')

In [167]:
def test_focus_conservation(z,nb_steps=10):
    '''
    '''
    lposz = []
    lposz_corr = []
    loptim = []
    lsurf0 = []
    ldiff2 = []
    step = 5
    delta_mod = 10   # dist from one model to the other.. 
    for j in range(nb_steps):
        print(f'######################## j is {j}')
        currz = z + (j-nb_steps//2)*step
        for i in range(3):
            ol.go_zpos( currz, move_type='d' )
        optim = find_z_optim(currz)
        loptim += [optim]
        ref = (len(list_mod)-1)//2
        posz_corr = currz-(optim-ref)*delta_mod #+ 2*(10-optim)
        print(f'##################### posz_corr = {posz_corr}')
        for i in range(3):
            ol.go_zpos( posz_corr, move_type='d' )
        take_pic_posz(3+j, posz_corr, bpp=8, move_type='d')
        lposz += [currz]
        lposz_corr += [posz_corr]
#         num_img = (3+j)+100
#         addr_img = f'bf_for_optimz{num_img}.tiff'
#         ev.take_pic(addr=addr_img, bpp=8, exp_time=100)
        surf0 = make_pred(3+j, mod=list_mod[6], thresh=127, show=False)
        lsurf0 +=[surf0]
        take_pic_posz(3+j+100, posz_corr + step, bpp=8, move_type='d')
        surf1 = make_pred(3+j+100, mod=list_mod[6], thresh=127, show=False)
        diff = surf1-surf0
        ldiff2 += [diff]
    lcurves = [lposz, lposz_corr, loptim, lsurf0, ldiff2, z, step]
    plot_curves(lcurves)

In [ ]:
test_focus_conservation(830230,nb_steps=30)

######################## j is 0
In go_zpos, val = 830155 
In go_zpos, val = 830155 
In go_zpos, val = 830155 
In go_zpos, val = 830155 
answ 0 12LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830175 
answ 0  2LOG +
2LOG
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is -3
for mod 5, diff is 0
for mod 6, diff is -1
for mod 7, diff is 2
for mod 8, diff is 2
for mod 9, diff is 4
for mod 10, diff is 0
for mod 11, diff is -3
for mod 12, diff is -17
max_surf = 25
***** optim is 0 *****
time elapsed is 3.2 s
##################### posz_corr = 830215
In go_zpos, val = 830215 
In go_zpos, val = 830215 
In go_zpos, val = 830215 
In go_zpos, val = 830215 
answ 0  2LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830220 
answ 0  2LOG !,E0251
apply the autocontrast..
######################## j is 1
In go_zpos, val = 830160 
In go_zpos, val = 830160 
In go_zpos, val = 830160 
In go_zpos, val = 830160 
a

for mod 8, diff is 7
for mod 9, diff is -4
for mod 10, diff is 3
for mod 11, diff is 1
for mod 12, diff is 127
max_surf = 528
***** optim is 0 *****
time elapsed is 3.1 s
##################### posz_corr = 830260
In go_zpos, val = 830260 
In go_zpos, val = 830260 
In go_zpos, val = 830260 
In go_zpos, val = 830260 
answ 0  2LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830265 
answ 0  2LOG !,E0251
apply the autocontrast..
######################## j is 10
In go_zpos, val = 830205 
In go_zpos, val = 830205 
In go_zpos, val = 830205 
In go_zpos, val = 830205 
answ 0 12LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830225 
answ 0  2LOG +
2LOG
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is 0
for mod 5, diff is 0
for mod 6, diff is 3
for mod 7, diff is 0
for mod 8, diff is 0
for mod 9, diff is 3
for mod 10, diff is 1
for mod 11, diff is 0
for mod 12, diff is 113
max_surf = 249
***** optim 

In go_zpos, val = 830250 
answ 0  2LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830270 
answ 0  2LOG !,E0251
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is 0
for mod 5, diff is 0
for mod 6, diff is 0
for mod 7, diff is 0
for mod 8, diff is -1
for mod 9, diff is -2
for mod 10, diff is 0
for mod 11, diff is 0
for mod 12, diff is 2
max_surf = 21
***** optim is 0 *****
time elapsed is 3.0 s
##################### posz_corr = 830310
In go_zpos, val = 830310 
In go_zpos, val = 830310 
In go_zpos, val = 830310 
In go_zpos, val = 830310 
answ 0 12LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830315 
answ 0  2LOG +
2LOG
apply the autocontrast..
######################## j is 20
In go_zpos, val = 830255 
In go_zpos, val = 830255 
In go_zpos, val = 830255 
In go_zpos, val = 830255 
answ 0  2LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 830275 
answ 0  2LOG +
2MOV
apply the autocontrast

## make the profiles with zstack

In [39]:
dic_surf = {}
loptim = []
nbstep = 40
for i in range(0,nbstep):  # 40
    print(f'############################## i = {i}')
    optim = find_z_optim(830361+(i-nbstep//2)*10,fill_dic_surf=True) # 833600
    loptim += [optim]
print('finished')

########################## i = 0
In go_zpos, val = 830161 
answ 0 V +
2LOG +

apply the autocontrast..
In go_zpos, val = 830181 
answ 0 2LOG !,E02511
apply the autocontrast..
--------
for mod 0, diff is -2366
for mod 1, diff is -1484
for mod 2, diff is -269
for mod 3, diff is -139
for mod 4, diff is 8
for mod 5, diff is 40
for mod 6, diff is 59
for mod 7, diff is 307
for mod 8, diff is 124
for mod 9, diff is 29
for mod 10, diff is 100
for mod 11, diff is -64
for mod 12, diff is 51
max_surf = 2371
***** optim is 0 *****
time elapsed is 3.3 s
########################## i = 1
In go_zpos, val = 830171 
answ 0 
2MOV +
2LO
apply the autocontrast..
In go_zpos, val = 830191 
answ 0 G +
2LOG !,E
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 3
for mod 3, diff is -18
for mod 4, diff is -2
for mod 5, diff is 3
for mod 6, diff is 0
for mod 7, diff is -4
for mod 8, diff is -5
for mod 9, diff is -6
for mod 10, diff is -6
for mod 11, diff is -23
for mod

for mod 2, diff is 102
for mod 3, diff is 9
for mod 4, diff is -11
for mod 5, diff is -86
for mod 6, diff is -18
for mod 7, diff is -59
for mod 8, diff is -36
for mod 9, diff is 17
for mod 10, diff is -11
for mod 11, diff is 18
for mod 12, diff is -14
max_surf = 879
***** optim is 3 *****
time elapsed is 2.9 s
########################## i = 16
In go_zpos, val = 830321 
answ 0 2LOG !,E02511
apply the autocontrast..
In go_zpos, val = 830341 
answ 0 
2MOV +
2LO
apply the autocontrast..
--------
for mod 0, diff is 1088
for mod 1, diff is 749
for mod 2, diff is 29
for mod 3, diff is 15
for mod 4, diff is -34
for mod 5, diff is -81
for mod 6, diff is -23
for mod 7, diff is -127
for mod 8, diff is -47
for mod 9, diff is -7
for mod 10, diff is -15
for mod 11, diff is -62
for mod 12, diff is -3
max_surf = 1216
***** optim is 0 *****
time elapsed is 2.9 s
########################## i = 17
In go_zpos, val = 830331 
answ 0 G +
2LOG !,E
apply the autocontrast..
In go_zpos, val = 830351 
answ 0 0251

for mod 0, diff is -111
for mod 1, diff is -414
for mod 2, diff is -730
for mod 3, diff is -799
for mod 4, diff is -633
for mod 5, diff is -1086
for mod 6, diff is -704
for mod 7, diff is -867
for mod 8, diff is -967
for mod 9, diff is -642
for mod 10, diff is -477
for mod 11, diff is -218
for mod 12, diff is -131
max_surf = 15872
***** optim is 0 *****
time elapsed is 2.9 s
########################## i = 31
In go_zpos, val = 830471 
answ 0 V +
2LOG +

apply the autocontrast..
In go_zpos, val = 830491 
answ 0 2LOG !,E02511
apply the autocontrast..
--------
for mod 0, diff is -78
for mod 1, diff is -274
for mod 2, diff is -419
for mod 3, diff is -567
for mod 4, diff is -496
for mod 5, diff is -793
for mod 6, diff is -633
for mod 7, diff is -889
for mod 8, diff is -901
for mod 9, diff is -621
for mod 10, diff is -546
for mod 11, diff is -513
for mod 12, diff is -302
max_surf = 15241
***** optim is 0 *****
time elapsed is 3.0 s
########################## i = 32
In go_zpos, val = 830481 
a

In [ ]:
dir(ol)

In [ ]:
ev.retrieve_autocontrast()

## plot the profiles and predictions.. 

In [41]:
allmax = 0
for k,v in dic_surf.items():
    print(k)
    maxsurf = max(dic_surf[k])
    if allmax<maxsurf:
        allmax=maxsurf
fact_norm = (len(list_mod)-1)/allmax
for mind, (k,v) in enumerate(dic_surf.items()):
    print(k)
    plt.plot(np.array(dic_surf[k])*fact_norm, alpha=0.3, label=f'lev{mind}')
# for pt in loptim:
#     plt.plot(pt,'rx')
plt.legend()
loptim.insert(0,0)
plt.plot(loptim,'rx')
    
plt.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
0
1
2
3
4
5
6
7
8
9
10
11
12


##  Controls

In [146]:
print(ol.ask_zpos())

830250


In [ ]:
for k,v in dic_surf.items():
    print(k)
    if k!='max':
        plt.plot(np.array(dic_surf[k]))

In [ ]:
find_z_optim(834570) # 833812

In [147]:
curr_pos = 830250
for i in range(3):
    ol.go_zpos( curr_pos, move_type='d' )

In go_zpos, val = 830250 
In go_zpos, val = 830250 
In go_zpos, val = 830250 


In [148]:
find_z_optim(curr_pos)

In go_zpos, val = 830250 
answ 0 
2LOG +
2MO
apply the autocontrast..
In go_zpos, val = 830270 
answ 0 V +
2LOG +

apply the autocontrast..
--------
for mod 0, diff is -670
for mod 1, diff is -769
for mod 2, diff is -492
for mod 3, diff is -402
for mod 4, diff is -494
for mod 5, diff is -441
for mod 6, diff is -281
for mod 7, diff is 134
for mod 8, diff is 621
for mod 9, diff is 1236
for mod 10, diff is 1166
for mod 11, diff is 1293
for mod 12, diff is 1421
max_surf = 13413
***** optim is 7 *****
time elapsed is 3.3 s


7

In [149]:
for i in range(3):
    take_pic_posz(301, curr_pos-10 , bpp=8, move_type='d')

In go_zpos, val = 830240 
answ 0 2LOG +
2MOV 
apply the autocontrast..
In go_zpos, val = 830240 
answ 0 +
2LOG +
2L
apply the autocontrast..
In go_zpos, val = 830240 
answ 0 OG +
2MOV +
apply the autocontrast..


In [156]:
def aa(ll):
    a, b, c = ll
    print(a,b,c)
aa([1,6,7])

1 6 7
